# 🧪 CI Local en Notebook (Windows/VS Code)

Este notebook ejecuta **lint (Ruff)**, **SAST (Bandit)**, **auditoría de dependencias (pip-audit)** y **tests con cobertura (≥80%)**
directamente desde Jupyter/VS Code en tu máquina Windows.

📌 **Ubícalo en la *raíz* del proyecto** (donde está `requirements.txt`).

Si no tienes las herramientas instaladas, el notebook las instalará según tu `requirements.txt`.


## 0) Configuración
Puedes desactivar temporalmente la auditoría de dependencias (pip-audit) si te está bloqueando.


In [ ]:
# Cambia a True si quieres saltar pip-audit en este run
SKIP_AUDIT = False

PROJECT_ROOT = '.'  # carpeta raíz del repo (este notebook debe estar ahí)
REQ_FILE = 'requirements.txt'

print('skip audit?:', SKIP_AUDIT)


## 1) Instalar dependencias (según `requirements.txt`)
Usa el entorno actual de Jupyter/VS Code. Si estás en un **venv**, quedará instalado ahí.


In [ ]:
import os, sys, subprocess, shlex, pathlib
req = pathlib.Path(REQ_FILE)
if not req.exists():
    raise FileNotFoundError('No se encontró requirements.txt. Coloca el notebook en la RAÍZ del proyecto.')

print('Python:', sys.executable)
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', str(req)])
print('Dependencias instaladas.')


## 2) Lint — Ruff
Falla si encuentra errores de estilo serios.


In [ ]:
import sys, subprocess
try:
    subprocess.check_call([sys.executable, '-m', 'ruff', 'check', 'app'])
    print('Ruff OK')
except subprocess.CalledProcessError as e:
    raise SystemExit('Ruff encontró problemas. Revisa la salida anterior.')


## 3) SAST — Bandit
Análisis estático de seguridad sobre el código.


In [ ]:
import sys, subprocess
try:
    subprocess.check_call([sys.executable, '-m', 'bandit', '-q', '-r', 'app', '-x', 'tests'])
    print('Bandit OK')
except subprocess.CalledProcessError:
    raise SystemExit('Bandit encontró problemas. Revisa la salida anterior.')


## 4) Auditoría de dependencias — pip-audit *(opcional)*
Puedes desactivarla con `SKIP_AUDIT = True` si te bloquea por vulnerabilidades de terceros.


In [ ]:
import sys, subprocess
if SKIP_AUDIT:
    print('Saltando pip-audit por configuración SKIP_AUDIT=True')
else:
    try:
        subprocess.check_call([sys.executable, '-m', 'pip_audit', '--strict'])
        print('pip-audit OK')
    except subprocess.CalledProcessError:
        raise SystemExit('pip-audit encontró vulnerabilidades. Ajusta dependencias o ejecuta con SKIP_AUDIT=True.')


## 5) Tests + cobertura (≥80%)
Genera reporte HTML en `coverage_html/index.html` y falla si la cobertura es menor a 80%.


In [ ]:
import sys, subprocess
try:
    # Usamos la configuración de pytest.ini del proyecto
    subprocess.check_call([sys.executable, '-m', 'pytest'])
    print('PyTest OK — Cobertura OK (>=80%).')
except subprocess.CalledProcessError:
    raise SystemExit('PyTest o cobertura fallaron. Revisa la salida para ver detalles.')


## 6) Abrir reporte de cobertura (HTML)
Este paso imprime la ubicación del reporte y, si estás en VS Code/Jupyter, puedes abrirlo desde el explorador de archivos.


In [ ]:
import pathlib, webbrowser
html = pathlib.Path('coverage_html/index.html')
print('Reporte de cobertura:', html.resolve())
if html.exists():
    print('Puedes abrirlo manualmente en tu navegador.')
else:
    print('Aún no existe. Asegúrate de que PyTest generó coverage_html (ver pytest.ini).')


---
### Notas
- Este notebook **no despliega** Docker; la API local de FastAPI sigue disponible con tus tareas de VS Code o `docker compose up`.
- La documentación de tu API continúa en **http://localhost:8000/docs**.
- Este pipeline local refleja lo del CI de GitHub: lint → SAST → audit → tests+cobertura.
